In [14]:
class KnowledgeBase:
    def __init__(self):
        self.rules = ["¬P1,1",
    "B1,1 ⇔ (P1,2 ∨ P2,1)",
    "B2,1 ⇔ (P1,1 ∨ P2,2 ∨ P3,1)",
    "¬B1,1",
    "B2,1"] #Reglas iniciales, se pueden establecer para todas las casillas
        self.sentences = []

    def tell(self, sentence):
        self.sentences.append(sentence) #Agregar oraciones

    def ask(self, query): #action, 0
        # Aquí podrías implementar un mecanismo más complejo para la inferencia,
        # como resolución o encadenamiento hacia adelante/atrás.

        for sentence in reversed(self.sentences):
            if query in sentence:
                return sentence[0]
        print("frase en analisis: ", self.sentences[-1])
        if "G" == str(self.sentences[-1])[2:4]:#if 'Glitter' in self.sentences[-1], preguntar si hay percepción Glitter
            return 'Grab'
        elif "St" in str(self.sentences[-1])[2:5] and agent.has_arrow:
            return 'Shoot'
        elif "Br" in str(self.sentences[-1])[2:5]:
            return 'TurnLeft'  # Ejemplo simple
        else:
            return 'Forward'

def extract_symbols(KB,alpha): # La función extract simbolos nos permite extraer los simbolos relevantes del conocimiento base y de alpha
  propotitions=[]
  propotitions.append(alpha)
  for i in agent.kb.rules:
    if "Executed" not in i:
      if type(i)==tuple and len(i[0])!=0:
        print(i[0])
        propotitions.append(str(i[0][0]))
      elif (type(i)!=tuple): propotitions.append(i)
  symbols = set()
  print(propotitions)
  for sentence in propotitions:
      # Divide y extrae las partes de la oración, aquí se hace de manera simplificada
      for symbol in sentence.replace("(", "").replace(")", "").replace("⇔", "").replace("¬", "").split():
          if symbol not in ["∨", "∧", "⇒"]:
              symbols.add(symbol)
  return(list(symbols))

def tt_entails(KB, alpha):
    """Verifica si KB lógicamente implica alpha usando la enumeración de la tabla de verdad"""
    symbols = extract_symbols(KB, alpha)  # Extraer los símbolos del KB y alpha
    #print(symbols)
    return tt_check_all(KB, alpha, symbols, {})

def tt_check_all(KB, alpha, symbols, model):
    """Función recursiva que evalúa si el modelo satisface el KB y alpha."""
    if not symbols: #en caso de que ya se halla asignado valor True o False a todos los simbolos
        print("Probar con", model)
        if pl_true_kb(KB, model): #ver si el modelo satisface el KB
            print("Es el modelo de prueba que cumple con KB: ", model)
            print("Esto es alpha: ", alpha)
            print("Esto es el resultado de alpha en el modelo: ", pl_true(alpha, model))
            return pl_true(alpha, model)
        else:
            return True  # Si KB es falso en el modelo, no importa alpha
    else:
        P = symbols[0]  # El primer símbolo
        #print(P)
        rest = symbols[1:]  # Los símbolos restantes
        # Probar con P verdadero
        model_true = model.copy()
        model_true[P] = True
        # Probar con P falso
        model_false = model.copy()
        model_false[P] = False
        #print("Probar con P False", model_false)
        # Evaluar recursivamente ambos casos
        return tt_check_all(KB, alpha, rest, model_true) and tt_check_all(KB, alpha, rest, model_false)



def pl_true_kb(KB, model):
    #Evalúa si todo el conocimiento base es verdadero en un modelo dado.
    for sentence in KB:
        print("resultado de evaluación de kb en el modelo: ", pl_true(sentence, model))
        if not pl_true(sentence, model):
            return False  # Si alguna oración es falsa, todo el KB es falso
    return True  # Si todas las oraciones son verdaderas, el KB es verdadero

def pl_true(sentence, model):
    # Elimina espacios para simplificar
    sentence = sentence.replace(")", "").replace("(", "").replace(" ","")

    # Caso para la negación (¬)
    if sentence.startswith("¬"):
        return not pl_true(sentence[1:], model)

    # Caso para la bicondicional (⇔)
    if "⇔" in sentence:
        lhs, rhs = sentence.split("⇔")
        #print("xxx: ", pl_true(lhs, model))
        #print("yyy: ", pl_true(rhs, model))
        return pl_true(lhs, model) == pl_true(rhs, model)

    # Caso para el condicional (⇒)
    if "⇒" in sentence:
        lhs, rhs = sentence.split("⇒")
        return not pl_true(lhs, model) or pl_true(rhs, model)

    # Caso para la disyunción (∨)
    if "∨" in sentence:
      if len(sentence.split("∨"))==2:
        lhs, rhs = sentence.split("∨")
        #print("esto es lhs: ", lhs)
        #print("esto es rhs: ", rhs)
        return pl_true(lhs, model) or pl_true(rhs, model)
      else:
        lhs, rhs, ohs = sentence.split("∨")
        return pl_true(lhs, model) or pl_true(rhs, model) or pl_true(ohs, model)

    # Caso para la conjunción (∧)
    if "∧" in sentence:
        lhs, rhs = sentence.split("∧")
        return pl_true(lhs, model) and pl_true(rhs, model)

    # Si es una proposición atómica, busca en el modelo
    if sentence in model:
        return model[sentence]

    # Retorna False si no encuentra la proposición en el modelo
    return False



class WumpusAgent:
    def __init__(self):
        self.kb = KnowledgeBase()
        self.position = (1,1)
        self.orientation = 'EAST'
        self.has_gold = False
        self.has_arrow = True
        self.t = 0  # Inicializar el contador de tiempo

    def make_percept_sentence(self, percept, t):
      propotionalPercept = [] #crea la oracion prposisional del percepto
      for i in range(len(percept)-1):
          if 'Glitter' == percept[i]:
              propotionalPercept.append('G'+str(self.position)[1:5].replace(" ","")) # Gx,y
          elif 'Stench' == percept[i]:
              propotionalPercept.append('St'+str(self.position)[1:5].replace(" ","")) # Sx,y
          elif 'Brezze' == percept[i]:
              propotionalPercept.append('Br'+str(self.position)[1:5].replace(" ","")) # Brx,y
          elif 'Bump' == percept[i]:
              propotionalPercept.append('Bu'+str(self.position)[1:5].replace(" ","")) # Bux,y
          elif 'Scream' == percept[i]:
              propotionalPercept.append('Sc'+str(self.position)[1:5].replace(" ","")) # Scx,y
      return ''.join(propotionalPercept), t  # Convierte la lista en una cadena concatenada en el segundo percepto devuelve Br2,1 1



    def make_action_query(self, t):
        return f"Action, {t}"

    def make_action_sentence(self, action, t):
        return f"Executed({action}, {t})" #Crear Oración de Action

    def act(self, percept):
        # Actualización de la percepción
        print(self.kb.sentences)
        percept_sentence = self.make_percept_sentence(percept, self.t)
        self.kb.tell(percept_sentence)

        # Consulta qué acción tomar basada en el percept
        action = self.kb.ask(self.make_action_query(self.t))

        action_sentence = self.make_action_sentence(action, self.t)
        self.kb.tell(action_sentence)
        self.execute_action(action)

        # Incrementar el contador de tiempo
        self.t += 1

        return action

    def execute_action(self, action):
        if action == 'Forward':
            self.move_forward()
        elif action == 'TurnLeft':
            self.turn_left()
        elif action == 'TurnRight':
            self.turn_right()
        elif action == 'Grab':
            self.has_gold = True
        elif action == 'Shoot':
            self.has_arrow = False

    def move_forward(self):
        x, y = self.position
        if self.orientation == 'EAST':
            self.position = (x + 1, y)
        elif self.orientation == 'WEST':
            self.position = (x - 1, y)
        elif self.orientation == 'NORTH':
            self.position = (x, y + 1)
        elif self.orientation == 'SOUTH':
            self.position = (x, y - 1)

    def turn_left(self):
        orientations = ['NORTH', 'WEST', 'SOUTH', 'EAST']
        idx = orientations.index(self.orientation)
        self.orientation = orientations[(idx + 1) % 4]

    def turn_right(self):
        orientations = ['NORTH', 'EAST', 'SOUTH', 'WEST']
        idx = orientations.index(self.orientation)
        self.orientation = orientations[(idx + 1) % 4]

# Simulación simple del entorno y agente
agent = WumpusAgent()
alpha = "¬P1,2"
#symbols = ["Br1,1","Br2,1","P1,1","P1,2","P2,1","P2,2","P3,1"]


percepts = [
    ['None', 'None', 'None', 'None', 'None'],  # Inicial [Stench,Glitter,Brezze,Bump,Scream]
    ['None', 'None', 'Brezze', 'None', 'None']  # Cerca del Wumpus
]

for percept in percepts:
    action = agent.act(percept)
    print(f"Agente toma acción: {action}, posición: {agent.position}, orientación: {agent.orientation}, tiene oro: {agent.has_gold}")
print("EVALUACIÓN DE ALPHA: ", tt_entails(agent.kb.rules, alpha))

[]
frase en analisis:  ('', 0)
Agente toma acción: Forward, posición: (2, 1), orientación: EAST, tiene oro: False
[('', 0), 'Executed(Forward, 0)']
frase en analisis:  ('Br2,1', 1)
Agente toma acción: TurnLeft, posición: (2, 1), orientación: NORTH, tiene oro: False
['¬P1,2', '¬P1,1', 'B1,1 ⇔ (P1,2 ∨ P2,1)', 'B2,1 ⇔ (P1,1 ∨ P2,2 ∨ P3,1)', '¬B1,1', 'B2,1']
Probar con {'P2,1': True, 'P2,2': True, 'B2,1': True, 'P3,1': True, 'P1,2': True, 'B1,1': True, 'P1,1': True}
resultado de evaluación de kb en el modelo:  False
Probar con {'P2,1': True, 'P2,2': True, 'B2,1': True, 'P3,1': True, 'P1,2': True, 'B1,1': True, 'P1,1': False}
resultado de evaluación de kb en el modelo:  True
resultado de evaluación de kb en el modelo:  True
resultado de evaluación de kb en el modelo:  True
resultado de evaluación de kb en el modelo:  False
Probar con {'P2,1': True, 'P2,2': True, 'B2,1': True, 'P3,1': True, 'P1,2': True, 'B1,1': False, 'P1,1': True}
resultado de evaluación de kb en el modelo:  False
Probar c

ACTIVIDAD

1. Modifique el codigo anterior de forma que el conocimiento de los perceptos ("¬B1,1", "B2,1"), inicialice en sentences y sea coherente con el movimiento del agente. la inferencia debe seguir funcionando de forma adecuada

2. Pruebe otras oraciones de alpha con el fin de establecer si pueden ser inferidas a partir del KB, tiene sentido logico para usted dicha inferencia?

3. **Diagnóstico Médico con Lógica Proposicional y Verificación de Modelos**

Diseñar un agente lógico que realice diagnósticos médicos basados en síntomas percibidos, utilizando lógica proposicional y el algoritmo de verificación de modelos (TT-Entails?).

Reglas del Problema
Se definen los siguientes síntomas (S) y enfermedades (E), junto con sus reglas lógicas:

S1 : Fiebre →𝐸1:  Posible infección

Fiebre⇒Infeccion

𝑆2: Tos → 𝐸2 : Posible resfriado o infección respiratoria

Tos⇒Resfriado∨Infeccion_Respiratoria

𝑆3: Dolor de cabeza → 𝐸3: Posible migraña o E1: infección

Dolor_de_cabeza⇒Migrana∨Infeccion


𝑆4: Dolor de garganta → 𝐸4:: Posible resfriado o infección de garganta

Dolor_de_garganta⇒Resfriado∨Infeccion_de_garganta

Funciones a Implementar
TELL(KB, percepto):

Recibe un nuevo síntoma (percepto) y lo añade a la base de conocimiento (KB).
ASK(KB, consulta):

Utiliza el algoritmo de verificación de modelos (TT-Entails?) para verificar si la KB implica la enfermedad consultada.
Tareas para los estudiantes
Definir las reglas en lógica proposicional

Representar los síntomas y enfermedades como variables proposicionales.
Escribir las reglas de inferencia usando operadores lógicos (∧, ∨, ¬, →).
Implementar el algoritmo TT-Entails?

Comprobar si KB ⊨ α, es decir, si la base de conocimiento implica una enfermedad específica.
Simular el diagnóstico para tres pacientes:

Cada paciente debe presentar un conjunto de síntomas.
Se debe evaluar qué enfermedades son inferidas por el sistema lógico.